In [1]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'caret' was built under R version 3.6.3"
Loading required package: lattice

Warning message:
"package 'lattice' was built under R version 3.6.2"
Loading required package: ggplot2

Warning message:
"package 'tidyverse' was built under R version 3.6.3"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v tibble  3.1.0     v purrr   0.3.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 3.6.2"
Warning message:
"package 'tidyr' was built under R version 3.6.3"
Warning message:
"package 'readr' was built under R version 3.6.3"
Warning message:
"pack

In [2]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [3]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [4]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    401     402 

## Models

In [5]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [6]:

model_monmlp <- train(x=TrainData, y=TrainClasses, 
               method = "monmlp") 

** Ensemble 1 
Complex eigenvalues found for method = BFGS 
coefficients for function value 0.2323443  :
  [1]  4.946079e+01 -9.592261e+01 -2.430195e+01 -7.182631e+01 -9.795976e+01
  [6]  8.906770e+01  1.041330e+02  1.904500e+01 -1.902784e+01 -2.360031e+01
 [11]  1.011184e+02  5.471877e+01  6.263779e+01 -7.772188e+01  2.516229e+01
 [16] -2.507071e+01  5.030811e+01  3.527593e+01 -2.428914e+01 -3.565804e+01
 [21] -2.217379e+00  2.286701e+00 -1.961687e+01 -1.953596e+01  4.710102e+01
 [26]  2.473640e+01 -2.470784e+01 -5.554915e-02  3.555039e+01 -1.930101e+01
 [31] -3.313431e+01 -9.609072e+01  2.947770e+01 -3.573700e+01  3.574085e+01
 [36] -9.593380e+00  9.582690e+00  1.359841e+01 -1.352886e+01  3.446376e+01
 [41]  1.694043e+01 -1.815760e+01 -3.107406e+01 -7.151863e+01  1.720440e+01
 [46] -1.720726e+01 -1.022596e+02 -9.489813e+01 -1.546414e+02 -3.158727e+01
 [51] -1.458022e+01  4.847123e+01 -5.677905e+01  8.849077e+01  6.709663e+01
 [56]  1.923621e+01  2.421829e+01  1.213536e+02  3.444577e+

In [7]:

model_mlpW <- train(x=TrainData, y=TrainClasses, 
               method = "mlpWeightDecay") 

## Results

In [8]:
print(model_monmlp)
plot(model_monmlp,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Monotone Multi-Layer Perceptron Neural Network 

803 samples
180 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  hidden1  Accuracy   Kappa    
  1        0.7084094  0.4172001
  3        0.7053318  0.4118011
  5        0.6845372  0.3691125

Tuning parameter 'n.ensemble' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were hidden1 = 1 and n.ensemble = 1.


In [9]:
print(model_mlpW)
plot(model_mlpW,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Multi-Layer Perceptron 

803 samples
180 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.7331678  0.4667319
  1     1e-04  0.7216813  0.4449198
  1     1e-01  0.5037751  0.0000000
  3     0e+00  0.7394994  0.4790392
  3     1e-04  0.7369581  0.4741819
  3     1e-01  0.5048929  0.0000000
  5     0e+00  0.7397350  0.4792113
  5     1e-04  0.7421976  0.4837849
  5     1e-01  0.5031263  0.0000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 1e-04.


In [10]:
# collect resamples
results <- resamples(list(monmlp=model_monmlp,  mlpWeightDecay=model_mlpW))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: monmlp, mlpWeightDecay 
Number of resamples: 25 

Accuracy 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.6699029 0.6918033 0.7055016 0.7084094 0.7275986 0.7668919    0
mlpWeightDecay 0.6870748 0.7222222 0.7412587 0.7421976 0.7620690 0.7859649    0

Kappa 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.3399095 0.3821968 0.4101693 0.4172001 0.4556377 0.5310864    0
mlpWeightDecay 0.3764869 0.4400274 0.4844085 0.4837849 0.5238209 0.5715975    0


## Prediction

In [11]:
pred_mlpW <- predict(model_mlpW,TestData)
pred_mlpW.prob <- predict(model_mlpW,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_mlpW) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      61      60
   NotFast   39     184
                                          
               Accuracy : 0.7122          
                 95% CI : (0.6612, 0.7595)
    No Information Rate : 0.7093          
    P-Value [Acc > NIR] : 0.47963         
                                          
                  Kappa : 0.3429          
                                          
 Mcnemar's Test P-Value : 0.04442         
                                          
            Sensitivity : 0.6100          
            Specificity : 0.7541          
         Pos Pred Value : 0.5041          
         Neg Pred Value : 0.8251          
             Prevalence : 0.2907          
         Detection Rate : 0.1773          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6820          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6100000            0.7540984            0.5041322 
      Neg Pred Value            Precision               Recall 
           0.8251121            0.5041322            0.6100000 
                  F1           Prevalence       Detection Rate 
           0.5520362            0.2906977            0.1773256 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6820492

In [12]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_mlpW))
roc_knn


Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_mlpW))

Data: as.numeric(pred_mlpW) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6646

In [13]:
roc_obj

ERROR: Error in eval(expr, envir, enclos): oggetto "roc_obj" non trovato


In [ ]:
pred_svmMlpW.prob